# bert test

This is a text cell. Start editing!

In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import org.apache.spark.ml.{Pipeline, PipelineModel, Transformer}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.storage._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification._
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.param.{Param, ParamMap}
import org.apache.spark.ml.util.{DefaultParamsReadable, DefaultParamsWritable, Identifiable}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{col, explode, udf}
import org.apache.spark.sql.types.{DataTypes, StructType}
import scala.collection.mutable.WrappedArray



In [2]:
val bert = BertEmbeddings.pretrained(name="bert_multi_cased", lang="xx")

bert_multi_cased download started this may take some time.
Approximate size to download 638.7 MB
Download done! Loading the resource.


In [3]:
val sentenceStartTokenId = 101
val sentenceEndTokenId = 102

In [4]:
val docs = Seq(Array(123,119, 11025), Array(10135))
val prep = docs.map(tokens => {
    Array(sentenceStartTokenId) ++
      tokens ++
      Array(sentenceEndTokenId) ++
      Array.fill(bert.getMaxSentenceLength - tokens.length - 2)(0)
})
prep

List([I@29179b57, [I@6c22790f)

In [5]:
val tfBert = bert.getModelIfNotSet
tfBert

com.johnsnowlabs.ml.tensorflow.TensorflowBert@47cc886b

In [6]:
// Code to use embeddings of bert
tfBert.tag(prep, "module/bert/encoder/Reshape_1:0", bert.getMaxSentenceLength)

List([[F@7cf43265, [[F@1998d100)

In [7]:
val trainDf = Seq(0 to 100:_*).toDF()
    .withColumn("tokens", when(rand() > 0.2, Array(101, 123,119, 11025, 102)).otherwise(Array(101, 10135, 102)))

In [8]:
// val average_embeddings = 
val udf_aggBert = udf[Array[Float], WrappedArray[Int]](tokens => {
            val numTokens = tokens.length
            val input = Seq(tokens.toArray ++ Array.fill(bert.getMaxSentenceLength - numTokens)(0))
            val embeddings = tfBert.tag(input, "module/bert/encoder/Reshape_1:0", bert.getMaxSentenceLength)
            val sumEmbeddings = embeddings.head.take(numTokens).transpose.map(_.sum)
            sumEmbeddings.map(x => (x * 1.0 / numTokens).toFloat)
})
trainDf.withColumn("embeddings", udf_aggBert('tokens)).show(false)

+-----+---------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------